In [12]:
import pandas as pd
from sqlalchemy import create_engine, URL, text
from tqdm import tqdm

In [13]:
url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    host='134.28.58.100',
    port=25432,
    database='Patstat',
)

In [14]:
engine = create_engine(url_object)

In [30]:
query_cpc = """
    CREATE TABLE temp_cpc_patstat AS
    SELECT appln_id, cpc_class_symbol
    FROM public.tls224_appln_cpc
    WHERE cpc_class_symbol LIKE 'Y02%%';
"""

In [31]:
# Obtain a connection from the engine
with engine.connect() as connection:
    # Execute the query to create the temporary table
    connection.execute(text(query_cpc))

    # Fetch data into a pandas DataFrame within the same connection session
    df_cpc_patstat = pd.read_sql_query("SELECT * FROM temp_cpc_patstat", connection)

In [32]:
# Group by appln_id and aggregate cpc_class_symbol
df_cpc_patstat_list = df_cpc_patstat.groupby('appln_id')['cpc_class_symbol'].progress_apply(list).reset_index()

In [34]:
# Cast appln_id to string
df_cpc_patstat_list['appln_id'] = df_cpc_patstat_list['appln_id'].astype(str)

In [35]:
# Delete everything after "." in appln_id
df_cpc_patstat_list['appln_id'] = df_cpc_patstat_list['appln_id'].str.split('.').str[0]

In [36]:
# Insert temporary table into database
df_cpc_patstat_list.to_sql('temp_cpc_patstat', engine, if_exists='replace', index=False)

543

In [37]:
query_appln = """
    SELECT 
        a.appln_id, 
        a.appln_auth, 
        a.granted,
        p.publn_nr,
        t.appln_title
    FROM 
        tls201_appln AS a
    JOIN 
        temp_cpc_epo AS c 
    ON 
        a.appln_id = CAST(c.appln_id AS DOUBLE PRECISION)
    LEFT JOIN public.tls211_pat_publn AS p ON a.appln_id = p.appln_id
    LEFT JOIN public.tls202_appln_title AS t ON a.appln_id = t.appln_id;
"""

In [38]:
# Run query for each appln_id in epo_appln_id_list
df_appln_cpc_patstat = pd.read_sql(query_appln, engine)

In [42]:
# Drop all rows where appln_auth not equal to EP
df_appln_cpc_ep = df_appln_cpc_patstat[df_appln_cpc_patstat['appln_auth'] == 'EP']

In [43]:
# Drop all rows where granted not equal to Y
df_appln_cpc_ep_granted = df_appln_cpc_ep[df_appln_cpc_ep['granted'] == 'Y']

In [47]:
# Cast column appln_id to string
df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)
df_appln_cpc_patstat['appln_id'] = df_appln_cpc_patstat['appln_id'].astype(str)

/tmp/ipykernel_2138098/1801136619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)


In [48]:
# Throw away everything after dot in appln_id
df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].str.split('.').str[0]
df_appln_cpc_patstat['appln_id'] = df_appln_cpc_patstat['appln_id'].str.split('.').str[0]

# Cast appln_id to string
df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)
df_appln_cpc_patstat['appln_id'] = df_appln_cpc_patstat['appln_id'].astype(str)

/tmp/ipykernel_2138098/1341935432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].str.split('.').str[0]
/tmp/ipykernel_2138098/1341935432.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appln_cpc_ep_granted['appln_id'] = df_appln_cpc_ep_granted['appln_id'].astype(str)


In [49]:
# Merge df_cpc_ep and df_appln_cpc_ep_granted on appln_id to get cpc_class_symbol for granted patents
df_appln_cpc_ep_granted = df_appln_cpc_ep_granted.merge(df_cpc_epo_list, on='appln_id', how='left')
df_appln_cpc_patstat = df_appln_cpc_patstat.merge(df_cpc_epo_list, on='appln_id', how='left')

In [61]:
# Throw away rows with complicate duplicates
df_appln_cpc_ep_granted = df_appln_cpc_ep_granted.drop_duplicates(subset=['appln_id', 'appln_auth', 'granted', 'publn_nr']).reset_index(drop=True)
df_appln_cpc_patstat = df_appln_cpc_patstat.drop_duplicates(subset=['appln_id', 'appln_auth', 'granted', 'publn_nr']).reset_index(drop=True)

In [65]:
# Save both dataframes to csv
df_appln_cpc_ep_granted.to_csv('/mnt/hdd01/PATSTAT Working Directory/cleantech_ep_granted.csv', index=False)
df_appln_cpc_patstat.to_csv('/mnt/hdd01/PATSTAT Working Directory/cleantech_patstat_all.csv', index=False)